In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from PIL import Image
import numpy as np
import pandas as pd

klasor_yolu = 'img_test/'  
dosya_isimleri = [dosya for dosya in os.listdir(klasor_yolu) if dosya.endswith('.jpg')]

hedef_boyut = (106, 106)

data = []
for dosya in dosya_isimleri:
    tam_yol = os.path.join(klasor_yolu, dosya)
    with Image.open(tam_yol) as img:
        img = img.convert('L').resize(hedef_boyut) 
        numpy_dizi = np.array(img)  
        data.append(numpy_dizi)

x_test = np.array(data)

csv_dosya_yolu = 'test_sol.csv' 
csv_matrisi = pd.read_csv(csv_dosya_yolu).values

y_test = csv_matrisi[:, 1:]
print('Load ok')

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.fc1 = nn.Linear(64 * 26 * 26, 128)  
        self.fc2 = nn.Linear(128, 37) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64 * 26 * 26)  # Flatten

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model_path = 'CNN_SGD_2_1.pth'
net = torch.load(model_path, map_location=torch.device('cpu'))  

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()
    def forward(self, predicted, actual):
        return torch.sqrt(self.mse(predicted, actual))

criterion = RMSELoss()

print('Finished Training')

net.eval()

num_slices = 100
slice_size = len(x_test) // num_slices
loss_list=[]
for i in range(num_slices):
    start_idx = i * slice_size
    end_idx = (i + 1) * slice_size if i < num_slices - 1 else len(x_test)
    
    x_test_slice = x_test[start_idx:end_idx]
    y_test_slice = y_test[start_idx:end_idx]
    
    x_test_tensor = torch.from_numpy(x_test_slice).float().unsqueeze(1)
    y_test_tensor = torch.Tensor(y_test_slice)
    
    with torch.no_grad():
        predictions = net(x_test_tensor)
    
    loss_fn = RMSELoss()
    
    # Loss hesaplayın
    loss = loss_fn(predictions, y_test_tensor)
    loss_list.append(loss.item())
    print(f"Slice {i + 1} Loss:", loss.item())
    
    
print(np.mean(loss_list))